In [1]:
###Required packages
import numpy as np
from hyperopt import hp, fmin, tpe
from scikits.odes.ode import ode
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
from pyomo.environ import *
import time
import random
from scipy.optimize import Bounds, minimize



In [2]:
def toy_model(t, y, ydot, params):
    kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467
    T = 1; E = 2; X = 2
    A, W = params
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    for e in range(E):
        ydot[e+X] = -lam*y[e+X] + np.sum(A[e]*np.array([activation(y[T], W[e][2], W[e][1], W[e][0]), repression(y[T], W[e][2], W[e][1], W[e][0]), W[e][2]]))
    ydot[E+X] = (Vin - y[X+1]*nonlinearity(y[X-1], kc, km))**2 #J1
    ydot[E+X+1] = np.sum([np.sum(A[e]*np.array([activation(y[T], W[e][2], W[e][1], W[e][0]), repression(y[T], W[e][2], W[e][1], W[e][0]), W[e][2]])) for e in range(E)]) #J2

def arch_converter(A):
    if A == 1:
        return ((0, 1, 0), (1, 0, 0))

    elif A == 2:
        return ((0, 0, 1), (0, 0, 1))

    elif A == 3:
        return ((0, 0, 1), (1, 0, 0))

    elif A == 4:
        return ((0, 1, 0), (0, 0, 1))
    else: return 'Invalid Circuit'

def name_converter(A):
    if A == ((0, 1, 0), (1, 0, 0)):
        return 'Dual Control'

    elif A == ((0, 0, 1), (0, 0, 1)):
        return 'No Control'

    elif A == ((0, 0, 1), (1, 0, 0)):
        return 'Downstream Activation'

    elif A == ((0, 1, 0), (0, 0, 1)):
        return 'Upstream Repression'
    else: return 'Invalid Circuit'

def loss_biological(j1, j2, alpha1=1E-5, alpha2=1E-2):
        """Computes scalarized loss including genetic constraint and product production"""
        loss = alpha1*j1 + alpha2*j2
        return j1, j2, loss

def activation(x, k, theta, n):
    return (k*(x/theta)**n)/(1+(x/theta)**n)

def repression(x, k, theta, n):
    return k/(1+(x/theta)**n)
    
def nonlinearity(x, kc, km):
    return (kc*x)/(km+x)

In [13]:

bounds = Bounds([1E-7, 1E-7, 0.001, 0.001], [0.001, 0.001, 10., 10.])

def ode_fun(x, function):
    k1, k2, theta1, theta2 = x
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    architecture = arch_converter(function)
    param_values = [[2., k1, theta1], [2., k2, theta2]]
    extra_options = {'old_api': False, 'user_data': [architecture, param_values], 'rtol':1E-4}
    ode_solver = ode('cvode', toy_model, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2, alpha1=1E-5, alpha2=1E-2)
    if (str(solution.flag) == 'StatusEnum.SUCCESS') and loss > 0.0: return loss
    else: return 1.E7

start_time = time.time()
function = 3
init_values = [np.random.uniform(1E-7, 1E-3), np.random.uniform(1E-7, 1E-3), np.random.uniform(0.001, 10.), np.random.uniform(0.001, 10.)]
sol = minimize(ode_fun, init_values, args=(function), method='TNC', bounds=bounds) #, options={'gtol':1e-10, 'stepmx':1E-2}
elapsed_time = time.time() - start_time
sol

     fun: 218.31181029745036
     jac: array([377666.16841017,      0.        ,  10769.56434349, -15333.0015105 ])
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 35
     nit: 1
  status: 2
 success: True
       x: array([2.21700200e-04, 8.70860261e-04, 6.91846921e-01, 3.33354365e-01])

In [20]:
solutions = []
times = []
for i in range(100):
    start_time = time.time()
    function = 3
    init_values = [np.random.uniform(1E-7, 1E-3), np.random.uniform(1E-7, 1E-3), np.random.uniform(0.001, 10.), np.random.uniform(0.001, 10.)]
    sol = minimize(ode_fun, init_values, args=(function), method='TNC', bounds=bounds) #, options={'gtol':1e-10, 'stepmx':1E-2}
    elapsed_time = time.time() - start_time
    times.append(elapsed_time)
    solutions.append(sol)


[CVODE ERROR]  CVode
  At t = 490.54, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 421.097, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 478.797, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 478.797, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 488.997, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 489.752, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 450.124, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 433.67, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 418.638, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 483.107, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 468.222, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 476.799, mxstep steps taken before reaching tout.


[CVODE ERROR]  CV

In [41]:
solutions = []
times = []
for j in [1, 2, 3, 4]:
    for i in range(100):
        start_time = time.time()
        function = j
        init_values = [np.random.uniform(1E-7, 1E-3), np.random.uniform(1E-7, 1E-3), np.random.uniform(0.001, 10.), np.random.uniform(0.001, 10.)]
        sol = minimize(ode_fun, init_values, args=(function), method='TNC', bounds=bounds) #, options={'gtol':1e-10, 'stepmx':1E-2}
        elapsed_time = time.time() - start_time
        times.append(elapsed_time)
        solutions.append(sol)


[CVODE ERROR]  CVode
  At t = 51.6737, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 250.16, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 48.6915, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 428.321, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 428.321, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 428.321, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 340.704, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 340.704, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 340.704, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 419.781, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 419.781, mxstep steps taken before reaching tout.


[CVODE ERROR]  CVode
  At t = 419.781, mxstep steps taken before reaching tout.


[CVODE ERROR]  C

In [43]:
visualization_data = pd.DataFrame({'Method':[], 'Computational Time':[], 'Best Loss':[], 'Function Evaluations':[], 'Message':[]})
for s, t in zip(solutions, times):
    row = pd.DataFrame({'Method': ['Gradient-Based'], 'Computational Time': [t], 'Best Loss': [s.fun], 'Function Evaluations': [s.nfev], 'Message':[s.message]})
    visualization_data = pd.concat([visualization_data, row])
visualization_data.to_csv('nlp_100_total.csv')